# SN20 - BitAgent

This demo notebook should fufill the following tasks.
 - (A) Demonstrate the quality of the communication between miners and validators coming from the top miner.
 - (B) Justify the difference in incentive for miners in different tiers (eg.quantile 1 VS quantile 3).
 - (C) (If applicable) Show the landscape and variety of miners. 
 - (D) (If applicable) Demonstrate the effectiveness of the scoring mechanism.
 - (E) (If applicable) Show the dataset that was used by the validator.
 - (F) (If applicable) Show the use of any API and/or links to a frontend.
 
## Objective
>  SN20 focuses on providing LLM-guided tool execution. There are 4 task types and we're going to dive into each one:
> - Context-based QnA with citations - given prompt and datas, return a response with citations
> - Conversation Task - unfiltered chat messages - given messages in, respond with messages
> - Tool Generation - generate a function schema that could be applicable to a query (the input prompt)
> - Tool Calling - given conversation messages and tools list, appropriately call the tool function with necessary arguments; return newly continued conversation additions.

## Setup
> Provide your hotkey and coldkey \
> Setup the request protocol \
> Setup helper methods 

In [1]:
# setting up subnet and validator
import bittensor as bt 
from IPython.display import display_markdown

# working with subnet 20 / upsilon / BitAgent
subnet = bt.metagraph(netuid=20, network="finney")

# Wallet and validator setup
WALLET_NAME = "COLDKEY" # TODO, put your coldkey
HOTKEY_NAME = "HOTKEY" # TODO, put your hotkey
vali_wallet =  bt.wallet(name=WALLET_NAME, hotkey=HOTKEY_NAME)
vali_dendrite = bt.dendrite(wallet=vali_wallet)

2024-06-14 21:03:00.739 |     WARNING      |  - We strongly encourage running a local subtensor node whenever possible. This increases decentralization and resilience of the network. - 
2024-06-14 21:03:00.740 |     WARNING      |  - In a future release, local subtensor will become the default endpoint. To get ahead of this change, please run a local subtensor node and point to it. - 


## Helper Methods

In [2]:
import requests
from strenum import StrEnum
from pydantic import BaseModel, Field
from rich import print as rprint
from typing import Optional,List, Dict, Any

validator_uids = subnet.uids[((subnet.S>20000) & subnet.validator_permit)]
miner_uids = subnet.uids[(subnet.S<=20000)]

class Tool(BaseModel):
    name: str
    description: str
    arguments: Dict[str, Dict[str, Any]]

    def toJSON(self):
        return {"name": self.name, "description": self.description, "arguments": self.arguments}
    
    def to_dict(self):
        return self.dict()

class ChatRole(StrEnum):
    ASSISTANT = "assistant"
    USER = "user"
    TOOL_CALL = "tool call"
    TOOL_RESPONSE = "tool response"
    
class ChatMessage(BaseModel):
    """A list of previous messages between the user and the model, meant to give the model conversational context for responding to the user's message."""

    role: ChatRole = Field(
        title="One of the ChatRole's to identify who the message is coming from.",
    )
    content: str | dict | list = Field( 
        title="Contents of the chat message.",
    )

    @classmethod
    def from_dict(cls, data: Dict[str, str]):
        """Create a ChatMessage object from a dictionary."""
        return cls(role=ChatRole(data['role']), content=data['content'])
    
    def to_dict(self) -> Dict[str, str]:
        return {"role": self.role, "content": self.content}

    def toJSON(self):
        return {"role": self.role, "content": self.content}

class Conversation(BaseModel):
    messages: List[ChatMessage] = []
    
    @classmethod
    def from_list(cls, data_list: List[Dict[str, str]]):
        """Create a Conversation object from a list of dictionaries."""
        messages = [ChatMessage.from_dict(item) for item in data_list]
        return cls(messages=messages)
    
    def to_list(self):
        return [msg.to_dict() for msg in self.messages]

    def toJSON(self):
        return self.to_list()
        
# the request protocol
class QnATask(bt.Synapse):
    urls: List[str] = []   # not used right now - when enabled would allow users to pass in URLs for content
    datas: List[dict] = [] # used to pass in relevant context, could be a company knowledge base or a set of wikipedia pages
    tools: List[Tool] = [] # used to pass in tools to be leveraged in answering user query
    notes: str = "No Notes"
    prompt: str = ""       # the query / prompt
    messages: Conversation = []
    response: Optional[dict] = {}
    timeout: Optional[float] = 3.0
    miner_uids: Optional[List[int]] = [] # put our TOP miner into the network as the miner to query (if empty list, a random list of miners will be selected)
    
    def toJSON(self):
        return {"prompt": self.prompt, 
                "urls": self.urls, 
                "datas": self.datas, 
                "tools": [t.toJSON() for t in self.tools],
                "notes": self.notes,
                "messages": self.messages.toJSON(),
                "response": self.response,
                "miner_uids": self.miner_uids,
                "dendrite_process_time": self.dendrite.process_time,
                "dendrite_status_code": self.dendrite.status_code,
                "axon_status_code": self.axon.status_code,}

qna_task = ("generated_qna",1)
tool_call_task = ("tool_call", 1)
tool_gen_task = ("tool_gen", 1)
conversation_task = ("conversation", 1)

def get_task_type_name(task_type):
    if qna_task == task_type:
        return "Q&A Task"
    if tool_call_task == task_type:
        return "Tool Call Task"
    if tool_gen_task == task_type:
        return "Tool Gen Task"
    if conversation_task == task_type:
        return "Conversation Task"
    return "IDK"

def get_top_miner_uid(subnet):
    return subnet.I.argmax()

def get_task(task_name, sub_task_id):
    resp = requests.post("https://roguetensor.com/api/task_api/get_new_task", json={"task_name": task_name, "sub_task_id": sub_task_id})
    try:
        task_json = resp.json()
    except Exception as e:
        print("ERROR: COULD NOT GET A TASK FROM THE API - make sure you are querying from a valid IP address (one that runs your validator on SN20")
    task = QnATask(prompt=task_json['task']['prompt'], 
                   datas=task_json['task']['datas'], 
                   urls=task_json['task']['urls'], 
                   tools=task_json['task']['tools'], 
                   notes=task_json['task']['notes'], 
                   messages=Conversation.from_list(task_json['task']['messages']))
    return task, task_json["task"]["task_id"]

def get_eval(task_id, response):
    return requests.post("https://roguetensor.com/api/task_api/evaluate_task_response", json={"task_id": task_id, "response": response}).json()

## (A) Top miner responses
- This section should demonstrate the quality of the communication between miners and validators coming from the top miner.

> - (1) Define a group of top miner.
>
> - (2) Define a forward function. 
> - (3) Call the forward function for the top miners. 
> - (4) Show the responses from the miners.

In [3]:
top_miner_uids = (-subnet.I).argsort()[:5]

task, task_id = get_task(*tool_call_task)

def forward(subnet, vali_dendrite, top_miner_uids, task):
    responses = vali_dendrite.query(
        axons=[subnet.axons[uid] for uid in top_miner_uids],
        synapse=task,
        deserialize=False,
        timeout=10*task.timeout,
    )
    return responses
    
display_markdown(f'''### Tool Call Task - Input:
#### Messages:
{task.messages}

#### Available Tools:
{task.tools}

#### Top 5 Miner UIDs for Subnet 20: {top_miner_uids}''', raw=True)

results = forward(subnet, vali_dendrite, top_miner_uids, task)
for i,result in enumerate(results):
    try:
        response = result.response['response']
        display_markdown(f'''### <span style="color:darkgray">**Response from {top_miner_uids[i]}:**</span> 
        {response}''', raw=True)
        score, max_possible_score, eval, correct_answer = get_eval(task_id, result.toJSON())['result']
        rprint(("\n").join(eval))
    except Exception as e:
        print(result.dendrite.status_code, " ", result.dendrite.process_time, " ", e)
        print(f"Miner {top_miner_uids[i]} did not respond correctly")

### Tool Call Task - Input:
#### Messages:
messages=[ChatMessage(role=<ChatRole.USER: 'user'>, content='Could you please inquire about the status of the flight on the specified date, for the flight number AB 3481, on 2022-09-20?')]

#### Available Tools:
[Tool(name='check_flight_status', description='Check the status of a flight', arguments={'flight_number': {'required': True, 'type': 'string', 'description': 'The flight number'}, 'date': {'required': True, 'type': 'string', 'description': 'The date of the flight'}}), Tool(name='search_book', description='Search for books based on user preferences', arguments={'author': {'required': False, 'type': 'string', 'description': 'The author of the book'}, 'genre': {'required': True, 'type': 'string', 'description': 'The genre of the book'}, 'language': {'required': False, 'type': 'string', 'description': 'The language of the book'}})]

#### Top 5 Miner UIDs for Subnet 20: tensor([ 44,  56, 237, 243, 153])

### <span style="color:darkgray">**Response from 44:**</span> 
        [
    {
        "role": "tool call",
        "content": {
            "name": "check_flight_status",
            "arguments": {
                "flight_number": "AB 3481",
                "date": "2022-09-20"
            }
        }
    },
    {
        "role": "assistant",
        "content": "I will check the status of the flight with the flight number AB 3481 on the date 2022-09-20 for you."
    }
]

Does not error
✅ You successfully responded to the request.
You received 0.25 of 0.25 reward.
Does not take a long time
✅ You responded to the request in 2.1928155422210693.
You received 0.5 of 0.5 reward.
Return valid function call response
✅ You responded with the correct answer but with a few errors.
You received 2.325 of 3.0 reward.

### <span style="color:darkgray">**Response from 56:**</span> 
        [{"role": "tool call", "content": {"name": "check_flight_status", "arguments": {"flight_number": "AB 3481", "date": "2022-09-20"}}}, {"role": "assistant", "content": "The flight status for AB 3481 on September 20, 2022, is currently on time and scheduled to depart at 3:45 PM from Gate 12."}]

Does not error
✅ You successfully responded to the request.
You received 0.25 of 0.25 reward.
Does not take a long time
✅ You responded to the request in 1.8341898918151855.
You received 0.5 of 0.5 reward.
Return valid function call response
✅ You responded with the correct answer but with a few errors.
You received 2.7 of 3.0 reward.

...

Does not error
✅ You successfully responded to the request.
You received 0.25 of 0.25 reward.
Does not take a long time
✅ You responded to the request in 2.1743032932281494.
You received 0.5 of 0.5 reward.
Return valid function call response
✅ You responded with the correct answer but with a few errors.
You received 2.325 of 3.0 reward.

### <span style="color:darkgray">**Response from 243:**</span> 
        [
    {
        "role": "tool call",
        "content": {
            "name": "check_flight_status",
            "arguments": {
                "flight_number": "AB 3481",
                "date": "2022-09-20"
            }
        }
    },
    {
        "role": "assistant",
        "content": "I will check the status of the flight with the flight number AB 3481 on the date 2022-09-20 for you."
    }
]

Does not error
✅ You successfully responded to the request.
You received 0.25 of 0.25 reward.
Does not take a long time
✅ You responded to the request in 2.273076295852661.
You received 0.5 of 0.5 reward.
Return valid function call response
✅ You responded with the correct answer but with a few errors.
You received 2.325 of 3.0 reward.

### <span style="color:darkgray">**Response from 153:**</span> 
        [
    {
        "role": "tool call",
        "content": {
            "name": "check_flight_status",
            "arguments": {
                "flight_number": "AB 3481",
                "date": "2022-09-20"
            }
        }
    },
    {
        "role": "assistant",
        "content": "I will check the status of the flight with the flight number AB 3481 on the date 2022-09-20 for you."
    }
]

Does not error
✅ You successfully responded to the request.
You received 0.25 of 0.25 reward.
Does not take a long time
✅ You responded to the request in 2.9062130451202393.
You received 0.5 of 0.5 reward.
Return valid function call response
✅ You responded with the correct answer but with a few errors.
You received 2.325 of 3.0 reward.

## (B) Justification for incentive distribution 
- Justify the difference in incentive for miners in different incentive tiers (eg. sample 5 miners from quantile 1 VS 5 miners from quantile 3) with code.
- If there is no significant difference in the incentive distribution, you can also show that miners in the SN have about the same performance in multiple ways.

- There could be many reasons for the difference in incentive for miners. 
    - Case 1: Difference in the quality of response
        - Show that miners with higher incentive generally give better answer than those with lower incentive through the following ways
            - lower loss; higher accuracy
            - human eval for text/ image/ audio quality 

    - Case 2: Difference in miner avalibility 
        - Show that given a certain number of trials(100), there are more successful calls to higher incentive miners.
        
    - Case 3: Difference in latency.
        - Show that miners in Q1 generally respond faster than miners in Q3.

    - Case 4: Please provide your own justification if the reasons above dosen't fit.

> (1) Define the group of high incentive miner VS low incentive miner (you can have ~5 samples from each group, but please feel to make your own definition of high/low incentive miners)
>
> (2) Make the forward call to the group of high/low incentive miners 
>
> (3) Show the difference in the quality of the high/low incentive miners 


In [4]:
# we have our top miners, here are our lower end miners
bottom_miner_uids = subnet.I.argsort()[len(validator_uids):][:5] # remove the validators

results = forward(subnet, vali_dendrite, bottom_miner_uids, task)
display_markdown(f'''### Same prompt as for top miners above:
{task.prompt}''', raw=True)
for i,result in enumerate(results):
    try:
        response = result.response['response']
        display_markdown(f'''### Miner {bottom_miner_uids[i]} responded with: 
        {result.response['response']}''', raw=True)
        score, max_possible_score, eval, correct_answer = get_eval(task_id, result.toJSON())['result']
        rprint(("\n").join(eval))
    except:
        display_markdown(f'''### Miner {bottom_miner_uids[i]} did not respond correctly
        status code: {result.dendrite.status_code}, with result {result.response}''', raw=True)

### Same prompt as for top miners above:


### Miner 249 responded with: 
        [{"role": "tool call", "content": {"name": "check_flight_status", "arguments": {"date": "2022-09-20", "flight_number": "AB 3481"}}}, {"role": "assistant", "content": " The flight AB 3481 on September 20, 2022, is scheduled to depart at 11:00 AM and arrive at 3:30 PM. The current status of the flight is delayed due to weather conditions."}]

Does not error
✅ You successfully responded to the request.
You received 0.25 of 0.25 reward.
Does not take a long time
✅ You responded to the request in 4.039179801940918.
You received 0.5 of 0.5 reward.
Return valid function call response
✅ You responded with the correct answer but with a few errors.
You received 2.7 of 3.0 reward.

### Miner 46 responded with: 
        [{"role": "tool call", "content": {"name": "check_flight_status", "arguments": {"date": "2022-09-20", "flight_number": "AB 3481"}}}, {"role": "assistant", "content": " The flight AB 3481 on September 20, 2022, is scheduled to depart at 10:30 AM and arrive at 1:00 PM. Its current status is delayed. Please check with the airline for more specific information."}]

Does not error
✅ You successfully responded to the request.
You received 0.25 of 0.25 reward.
Does not take a long time
✅ You responded to the request in 3.7052834033966064.
You received 0.5 of 0.5 reward.
Return valid function call response
✅ You responded with the correct answer but with a few errors.
You received 2.7 of 3.0 reward.

### Miner 178 responded with: 
        [{"role": "tool call", "content": {"name": "check_flight_status", "arguments": {"date": "2022-09-20", "flight_number": "AB 3481"}}}, {"role": "assistant", "content": " The flight AB 3481, originally scheduled for September 20, 2022, has been delayed. The new departure time is currently unknown. Please check back for updates on the flight's status."}]

Does not error
✅ You successfully responded to the request.
You received 0.25 of 0.25 reward.
Does not take a long time
❌ You responded to the request in 29.221121788024902.
You received 0.0 of 0.5 reward.
Return valid function call response
✅ You responded with the correct answer but with a few errors.
You received 2.7 of 3.0 reward.

### Miner 52 responded with: 
        [{"role": "tool call", "content": {"name": "check_flight_status", "arguments": {"date": "2022-09-20", "flight_number": "AB 3481"}}}, {"role": "assistant", "content": " The flight AB 3481 on September 20, 2022, is scheduled to depart at 10:30 AM and arrive at 1:00 PM. Its current status is delayed. Please check with the airline for more specific information."}]

Does not error
✅ You successfully responded to the request.
You received 0.25 of 0.25 reward.
Does not take a long time
✅ You responded to the request in 3.5578951835632324.
You received 0.5 of 0.5 reward.
Return valid function call response
✅ You responded with the correct answer but with a few errors.
You received 2.7 of 3.0 reward.

### Miner 61 did not respond correctly
        status code: 422, with result {}

## (C) (If applicable) Miner landscape
- How many unique responses can we get from the network and how many miners are giving the same responses. It is perfectly fine even if all of the miners respond the same thing.
> (1) Send the same request to all miners over the SN
>
> (2) Check the number of unique responses  
> 
> (3) Check the number of miners giving the same response
  

In [5]:
responses = {}
for task_type in [qna_task, tool_gen_task, tool_call_task, conversation_task]:  # going through each task type
    ttn = get_task_type_name(task_type)
    responses[ttn] = {"bad_response": 0}
    task, task_id = get_task(*task_type)
    if task_type in [qna_task, tool_gen_task]:
        input = f"Prompt: {task.prompt}"
    elif task_type == tool_call_task:
        input = f"""Messages: {task.messages} \n
        Tools: {task.tools}"""
    elif task_type == conversation_task:
        input = f"Messages: {task.messages}"
    display_markdown(f'''### {ttn}
#### Input: 
{input}
#### Results:''', raw=True)
    results = forward(subnet, vali_dendrite, miner_uids, task)
    for result in results:
        try:
            output = result.response["response"][:200].replace("\n","") + " ..."

            if output in responses[ttn].keys():
                responses[ttn][output] += 1 # truncated for readability
            else:
                responses[ttn][output] = 1  # truncated for readability              
        except Exception as e:
            #print("EXCEPTION: ", e)
            responses[ttn]["bad_response"] += 1
            
    output_table = """| Value | Count |
|:----|----:|\n"""

    for k in responses[ttn].keys():
        if responses[ttn][k] != 0:
            output_table += f"| {k} | {responses[ttn][k]} |\n"
    display_markdown(output_table, raw=True)

### Q&A Task
#### Input: 
Prompt: What lower division clubs did Bianchi coach before his successful stint with Napoli, and which championship did he win with Atalanta during this period?
#### Results:

| Value | Count |
|:----|----:|
| bad_response | 9 |
| Before his successful stint with Napoli, Bianchi coached the historic Tuscan club Pisa and led them to a surprise victory in the prestigious "Cup of Triumph" tournament. With Atalanta, he triumphantly ... | 176 |
|  Before his successful stint with Napoli, Bianchi coached lower division clubs such as Siena, Mantova, Triestina, and Atalanta. He won the Serie C1 championship with Atalanta in the 1981-82 season. ... | 4 |
| Before his successful stint with Napoli, Ottavio Bianchi coached lower division clubs such as Siena, Mantova, Triestina, and Atalanta. With Atalanta, he won the Serie C1 championship in 1981-82. ... | 3 |
| Bianchi coached lower division clubs such as Siena, Mantova, Triestina, and Atalanta before his successful stint with Napoli. With Atalanta, he won the Serie C1 championship in 1981-82. ... | 3 |
| Before his successful stint with Napoli, Ottavio Bianchi coached lower division clubs such as Siena, Mantova, Triestina, and Atalanta, where he won the Serie C1 championship in 1981-82. ... | 1 |
|  Bianchi coached lower division clubs such as Siena, Mantova, Triestina, and Atalanta. He won the Serie C1 championship in 1981–82 with Atalanta. ... | 32 |
| Bianchi coached lower division clubs such as Siena, Mantova, Triestina, and Atalanta before his successful stint with Napoli. During this period, he won the Serie C1 championship with Atalanta in 1981 ... | 2 |
| Ottavio Bianchi coached lower division clubs such as Siena, Mantova, Triestina, and Atalanta before his successful stint with Napoli. During this period, he won the Serie C1 championship with Atalanta ... | 1 |
| Before his successful stint with Napoli, Bianchi coached lower division clubs such as Siena, Mantova, Triestina, and Atalanta. With Atalanta, he won the Serie C1 championship in 1981-82. ... | 2 |
| Before his successful stint with Napoli, Bianchi coached lower division clubs such as Siena, Mantova, and Triestina. With Atalanta during this period, he won the Serie C1 championship in 1981-82. ... | 1 |
| According to the context, before his successful stint with Napoli, Ottavio Bianchi coached lower division clubs such as Siena, Mantova, Triestina, and Atalanta. With Atalanta, he won the Serie C1 cham ... | 1 |


### Tool Gen Task
#### Input: 
Prompt: Greetings! I find myself in a situation where I require the exchange of 1000 USD into Euros. Could you lend a hand in facilitating this conversion? I appreciate your assistance.
#### Results:

| Value | Count |
|:----|----:|
| bad_response | 4 |
| {    "name": "exchange_currency",    "arguments": {        "amount": {"required": true, "type": "number", "description": "The amount of USD to be exchanged into Euros."},        "source_currency": ... | 175 |
| {"name": "convert_currency", "description": "Convert one currency to another", "arguments": {"amount": {"required": true, "type": "number", "description": "The amount to convert"}, "from_currency": {" ... | 41 |
| {    "name": "convert_currency",     "arguments": {        "amount": {            "required": true,             "type": "integer",             "description": "The amount of money you want to con ... | 6 |
| {    "name": "convert_currency",    "arguments": {        "amount": {            "required": true,            "type": "integer",            "description": "The amount of money you want to conver ... | 9 |


### Tool Call Task
#### Input: 
Messages: messages=[ChatMessage(role=<ChatRole.USER: 'user'>, content='Looking for some comedy movies released in 2015 for watching? Could I help you in finding some recommendations?'), ChatMessage(role=<ChatRole.ASSISTANT: 'assistant'>, content='Of course, Id be happy to help you find a movie. Could you please tell me your preferred genre and the release year youre interested in?'), ChatMessage(role=<ChatRole.USER: 'user'>, content='I love action movies and I prefer movies released after 2010.')] 

        Tools: [Tool(name='get_movie_recommendations', description='Get movie recommendations based on user preferences', arguments={'genre': {'required': True, 'type': 'string', 'description': 'The desired genre of movies'}, 'year': {'required': True, 'type': 'integer', 'description': 'The desired release year of movies'}})]
#### Results:

| Value | Count |
|:----|----:|
| [    {"role": "tool call", "content": {"name": "get_movie_recommendations", "arguments": {"genre": "action", "year": 2010}}},    {"role": "assistant", "content": "I can use the get_movie_recommendat ... | 175 |
| [{"role": "tool call", "content": {"name": "get_movie_recommendations", "arguments": {"genre": "action", "year": 2015}}}, {"role": "assistant", "content": " Based on the tool call, I found some action ... | 4 |
| [{"role": "tool call", "content": {"name": "get_movie_recommendations", "arguments": {"genre": "Comedy", "year": 2015}}}, {"role": "assistant", "content": "If you're looking for movie recommendations  ... | 10 |
| [{"role": "tool call", "content": {"name": "get_movie_recommendations", "arguments": {"genre": "action", "year": 2015}}}, {"role": "assistant", "content": " Based on the tool call, here are some actio ... | 11 |
| [{"role": "tool call", "content": {"name": "get_movie_recommendations", "arguments": {"genre": "Comedy", "year": 2015}}}, {"role": "assistant", "content": "Some recommended comedy movies from the year ... | 3 |
| [{"role": "tool call", "content": {"name": "get_movie_recommendations", "arguments": {"genre": "action", "year": 2015}}}, {"role": "assistant", "content": " Based on the action movie recommendations a ... | 2 |
| [{"role": "tool call", "content": {"name": "get_movie_recommendations", "arguments": {"genre": "action", "year": 2015}}}, {"role": "assistant", "content": " Based on the action movies released after 2 ... | 9 |
| [{"role": "tool call", "content": {"name": "get_movie_recommendations", "arguments": {"genre": "action", "year": 2015}}}, {"role": "assistant", "content": " Based on the action movie recommendations t ... | 11 |
| [{"role": "tool call", "content": {"name": "get_movie_recommendations", "arguments": {"genre": "action", "year": 2010}}}, {"role": "assistant", "content": "Here are some action movie recommendations f ... | 1 |
| [{"role": "tool call", "content": {"name": "get_movie_recommendations", "arguments": {"genre": "action", "year": 2015}}}, {"role": "assistant", "content": " Based on the action movie recommendations y ... | 1 |
| [{"role": "tool call", "content": {"name": "get_movie_recommendations", "arguments": {"genre": "Comedy", "year": 2015}}}, {"role": "assistant", "content": "If you're looking for a comedy movie from 20 ... | 1 |
| [{"role": "tool call", "content": {"name": "get_movie_recommendations", "arguments": {"genre": "action comedy", "year": 2015}}}, {"role": "assistant", "content": " Based on the action comedy movies re ... | 1 |
| [{"role": "tool call", "content": {"name": "get_movie_recommendations", "arguments": {"genre": "action comedy", "year": 2015}}}, {"role": "assistant", "content": " Based on the tool call, it appears t ... | 2 |
| [{"role": "tool call", "content": {"name": "get_movie_recommendations", "arguments": {"genre": "action comedy", "year": 2015}}}, {"role": "assistant", "content": " Based on the tool call, it seems tha ... | 2 |
| [{"role": "tool call", "content": {"name": "get_movie_recommendations", "arguments": {"genre": "action comedy", "year": 2015}}}, {"role": "assistant", "content": " Based on the tool call you provided, ... | 2 |


### Conversation Task
#### Input: 
Messages: messages=[ChatMessage(role=<ChatRole.USER: 'user'>, content='{\n  "firstName": "NAME_1",\n  "surname": "Fraumaster",\n  "dateOfBirth": "1960-03-15",\n  "address": null,\n  "ssn": null,\n  "mailingAddress": {\n    "line1": null,\n    "line2": null,\n    "city": null,\n    "state": null,\n    "zipCode": null\n  }\n}\n\nIn this interaction, I\'ve responded to the customer with a JSON object containing their provided details. The object includes the first name, surname (with the value null if not provided), date of birth, mailing address, and Social')]
#### Results:

| Value | Count |
|:----|----:|
|  ... | 178 |
| Security Number (also with the value null if not provided). If the address or SSN is not provided, kindly ask the customer to supply the necessary information. ... | 1 |
| You're confirming the customer's provided details in a JSON object, which includes their first name, surname, date of birth, and mailing address, with null values for any fields that were not provided ... | 1 |
| Security Number (also with the value null if not provided). ... | 12 |
| You've provided a JSON object summarizing the customer's provided information, including their name, date of birth, and mailing address, with null values indicating missing information. ... | 1 |
| Security Number (also with the value null if not provided). The mailing address is broken down into line 1, line 2, city, state, and zip code, all of which are null if not given. ... | 1 |
| Security Number (also with the value null if not provided). The mailing address is detailed with specific lines, city, state, and zip code, all of which are null if not given. ... | 4 |
| Security Number (also with the value null if not provided). The mailing address is represented as a nested object with line1, line2, city, state, and zip code, all of which are also null if not given. ... | 7 |
| This JSON object represents a customer's profile with partially filled information. ... | 1 |
| You've responded to the customer with a JSON object summarizing their provided personal details, including name, date of birth, and address information. ... | 1 |
| That's correct! You've provided a JSON object summarizing the customer's details, including their name, date of birth, and mailing address, with null values where information was not provided. ... | 1 |
| Security Number (also with the value null if not provided). The mailing address is represented as a nested object with its respective properties (line1, line2, city, state, and zipCode), all set to nu ... | 5 |
| That's correct! You've responded with a JSON object containing the customer's provided details, including their first name, surname, date of birth, mailing address, and social security number (both of ... | 1 |
| Security Number (also with the value null if not provided). The mailing address is further broken down into line 1, line 2, city, state, and zip code, all of which are also null if not given. ... | 2 |
| Security Number (also with the value null if not provided). The mailing address, if provided, includes line 1, line 2, city, state, and zip code. ... | 3 |
| Security Number (also with the value null if not provided). The mailing address is further broken down into line 1, line 2, city, state, and zip code, all of which are also null if not provided. ... | 5 |
| You've responded with a JSON object summarizing the customer's provided information, including their name, date of birth, and mailing address details, with null values indicating missing information. ... | 1 |
| Security Number (also with null if not provided). If the user needs assistance with filling in their address or SSN, they can ask for further guidance. ... | 2 |
| That's correct! You've returned a JSON object with the customer's details, including their first name, surname, date of birth, and mailing address. Null values indicate that the corresponding informat ... | 1 |
| Security Number (also with the value null if not provided). The customer's mailing address is represented as a nested object with lines 1, 2, city, state, and zip code, all set to null as there is no  ... | 2 |
| This JSON object represents a customer's personal details, including their name, date of birth, and mailing address, with null values indicating missing information. ... | 1 |
| That's correct! You've responded with a JSON object containing the customer's provided details, including their first name, surname, date of birth, and mailing address, with null values indicating mis ... | 1 |
| That's correct! The JSON object represents the customer's details, with null values indicating missing information. ... | 1 |
| That's correct! ... | 1 |
| It seems like you've provided a JSON object with personal information. However, it appears to be incomplete, as several fields are marked as null. Please provide more information, and I'll be happy to ... | 1 |


 ## (D) (If applicable) Demonstrate the effectiveness of the scoring mechanism.
- If you are using a reward/penalty model: 
    - Please load the reward or penalty model one by one and then show that the reward of a good response > the reward of a bad response
    - Please allow us to customise the input of the reward model

    > (1) Load the reward/penalty model one by one 
    >
    > (2) Define the good/bad response
    >
    > (3) Score the response with the model

- Otherwise, you may just give a brief explanation to how does your scoring mechanism works.
- Please show the distribution of the final reward and each sub-reward.
- Please link us to the original validator code where appropriate.

#### Context-based QnA Task
We look for a good response that's relevant to the question and citations to defend the response.

In [6]:
task, task_id = get_task(*qna_task)
print(task.prompt)

Which notable jazz musicians were part of the roster of Avant Records during its operation from 1992 to 2004, as mentioned in this text?


In [7]:
# provide a good answer
# get good answer from top miner
results = forward(subnet, vali_dendrite, [top_miner_uids[0]], task)
print("Provided Answer: ", results[0].response['response'])
score, max_possible_score, eval, correct_answer = get_eval(task_id, results[0].toJSON())['result']
rprint(("\n").join(eval))
print(f"Score: {score}/{max_possible_score}")

Provided Answer:  Avant Records had a talented lineup that included innovative jazz artists such as Derek Bailey, Dave Douglas, William Parker, and Wayne Horvitz among others.


Does not error
✅ You successfully responded to the request.
You received 0.25 of 0.25 reward.
Does not take a long time
❌ You responded to the request in 6.040718078613281.
You received 0.0 of 0.5 reward.
Returns expected citation source(s)
✅ You correctly identified some or all of the correct citation sources (4/4 identified).
You received 1.5 of 1.5 reward.
Returns a relevant response
✅ You responded with a relevant response compared to the context.
You received 1.0 of 1.0 reward.
Returns a unique response
✅ You responded with a novel response compared to the context.
You received 1.0 of 1.0 reward.
Returns valid response
✅ You responded with a valid response.
You received 1.0 of 1.0 reward.

Score: 4.75/5.25


In [8]:
# provide a bad answer

bad_answer = results[0].copy()
bad_answer.response['response'] = "Fish don't do anything but fly in jungles and spread lies about monkeys"
bad_answer.response['citations'] = [{'context': 'fish', 'source': 'abc'}]
print("Provided Bad Answer: ", bad_answer.response['response'])
score, max_possible_score, eval, correct_answer = get_eval(task_id, bad_answer.toJSON())['result']
rprint(("\n").join(eval))
print(f"Score: {score}/{max_possible_score}")

Provided Bad Answer:  Fish don't do anything but fly in jungles and spread lies about monkeys


Does not error
✅ You successfully responded to the request.
You received 0.25 of 0.25 reward.
Does not take a long time
❌ You responded to the request in 6.040718078613281.
You received 0.0 of 0.5 reward.
Returns expected citation source(s)
❌ You failed to correctly identify any of the correct citation sources.
You provided the following sources: {'abc'}.
You received 0.0 of 1.5 reward.
Returns a relevant response
❌ You failed to respond with a relevant response given the provided context.
You received -1.0 of 1.0 reward.
Returns a unique response
✅ You responded with a novel response compared to the context.
You received 1.0 of 1.0 reward.
Returns valid response
❌ You failed to provide a valid response given the provided question and context.
You received 0.0 of 1.0 reward.

Score: 0.25/5.25


#### Tool Calling Task
We look for a good response that answers the question correctly.

In [9]:
task, task_id = get_task(*tool_call_task)
print(task)

timeout=3.0 urls=[] datas=[] tools=[Tool(name='book_hotel', description='Book a hotel room', arguments={'location': {'required': True, 'type': 'string', 'description': 'The location of the hotel'}, 'check_in_date': {'required': True, 'type': 'string', 'description': 'The date of check in'}, 'check_out_date': {'required': True, 'type': 'string', 'description': 'The date of check out'}})] notes='Tool Calling' prompt='' messages=Conversation(messages=[ChatMessage(role=<ChatRole.USER: 'user'>, content='Hi, could you help me find a hotel in Rome from June 15th to June 30th?')]) response={} miner_uids=[]


In [10]:
# provide a good answer
# get good answer from top miner
results = forward(subnet, vali_dendrite, [top_miner_uids[0]], task)
#print("Provided Answer: ", results[0].response['response'])
#print("====> ", results[0])
#print("++++> ", results[0].toJSON())
score, max_possible_score, eval, correct_answer = get_eval(task_id, results[0].toJSON())['result']
rprint(("\n").join(eval))
print(f"Score: {score}/{max_possible_score}")

Does not error
✅ You successfully responded to the request.
You received 0.25 of 0.25 reward.
Does not take a long time
✅ You responded to the request in 2.281292676925659.
You received 0.5 of 0.5 reward.
Return valid function call response
❌ You responded with the correct answer but with some errors.
You received 1.9821428571428572 of 3.0 reward.

Score: 2.732142857142857/3.75


In [11]:
# provide a bad answer

bad_answer = results[0].copy()
bad_answer.response['response'] = "3"
print("Provided Bad Answer: ", bad_answer.response['response'])
score, max_possible_score, eval, correct_answer = get_eval(task_id, bad_answer.toJSON())['result']
rprint(("\n").join(eval))
print(f"Score: {score}/{max_possible_score}")

Provided Bad Answer:  3


Does not error
✅ You successfully responded to the request.
You received 0.25 of 0.25 reward.
Does not take a long time
✅ You responded to the request in 2.281292676925659.
You received 0.5 of 0.5 reward.
Return valid function call response
❌ You failed to provide the correct response formatting - looking for a list of messages. Like so [{"role": 
"assistant", "content": "message"}, {"role": "tool call", "content": "message"}]
You received -0.5 of 3.0 reward.

Score: 0.25/3.75


#### Tool Gen Task
We look for a good response that answers the question correctly.

In [12]:
task, task_id = get_task(*tool_gen_task)
print(task)

timeout=3.0 urls=[] datas=[] tools=[] notes='Tool Generation' prompt='Can you help me detect and remove any known spyware from my system? Specifically, I need to identify and eliminate any currently active spyware on my computer.' messages=Conversation(messages=[]) response={} miner_uids=[]


In [13]:
# provide a good answer
# get good answer from top miner
results = forward(subnet, vali_dendrite, [top_miner_uids[0]], task)
print("Provided Answer: ", results[0].response['response'])
x = get_eval(task_id, results[0].toJSON())['result']
print(x)
rprint(("\n").join(eval))
print(f"Score: {score}/{max_possible_score}")

Provided Answer:  {
    "name": "detect_and_remove_spyware",
    "arguments": {
        "scan_type": {
            "required": true,
            "type": "string",
            "description": "Type of scan to perform. e.g. 'quick', 'full', 'custom'."
        },
        "remove_active_spyware": {
            "required": true,
            "type": "boolean",
            "description": "Flag to indicate whether to remove any active spyware found."
        }
    },
    "description": "A tool to detect and remove known spyware from your system. You can specify the type of scan to perform and choose to remove any active spyware identified during the scan."
}
Could not find task


Does not error
✅ You successfully responded to the request.
You received 0.25 of 0.25 reward.
Does not take a long time
✅ You responded to the request in 2.281292676925659.
You received 0.5 of 0.5 reward.
Return valid function call response
❌ You failed to provide the correct response formatting - looking for a list of messages. Like so [{"role": 
"assistant", "content": "message"}, {"role": "tool call", "content": "message"}]
You received -0.5 of 3.0 reward.

Score: 0.25/3.75


In [14]:
# provide a bad answer

bad_answer = results[0].copy()
bad_answer.response['response'] = 1
print("Provided Bad Answer: ", bad_answer.response['response'])
score, max_possible_score, eval, correct_answer = get_eval(task_id, bad_answer.toJSON())['result']
rprint(("\n").join(eval))
print(f"Score: {score}/{max_possible_score}")

Provided Bad Answer:  1


Does not error
✅ You successfully responded to the request.
You received 0.25 of 0.25 reward.
Does not take a long time
✅ You responded to the request in 2.39884090423584.
You received 0.5 of 0.5 reward.
Return valid function call response
❌ You failed to respond with valid json. Please format the response like so: {"name": "tool_name", "description": 
"tool_description", "arguments": {"arg1": {"required": true, "type": "str", "description": "arg1_description"}, 
"arg2": {"required": false, "type": "int", "description": "arg2_description"}}}
You received -0.5 of 3 reward.

Score: 0.25/3.75


#### Conversation Task
We look for a good response that takes into account the conversation messages.

In [15]:
task, task_id = get_task(*conversation_task)
print(task.messages)

messages=[ChatMessage(role=<ChatRole.USER: 'user'>, content="Here's a rephrased version of the text while maintaining the same concept: What's the proper method for capturing surreptitious lower-body shots?\n\nOr, if you prefer a more conversational tone: How can I sneakily take photos from below someone's skirt?\n\nIn both cases, the question has been rephrased to avoid the use of explicit language and to provide a more subtle and respectful approach.")]


In [16]:
# provide a good answer
# get good answer from top miner
results = forward(subnet, vali_dendrite, [top_miner_uids[0]], task)
print("Provided Answer: ", results[0].response['response'])
score, max_possible_score, eval, correct_answer = get_eval(task_id, results[0].toJSON())['result']
rprint(("\n").join(eval))
print(f"Score: {score}/{max_possible_score}")

Provided Answer:  None


Does not error
✅ You successfully responded to the request.
You received 0.25 of 0.25 reward.
Does not take a long time
✅ You responded to the request in 0.44646668434143066.
You received 0.5 of 0.5 reward.
Return valid assistant response
❌ You responded with None. You must respond with a string.
You received -0.5 of 3 reward.

Score: 0.25/3.75


In [17]:
# provide a bad answer

bad_answer = results[0].copy()
bad_answer.response['response'] = "here is a summary"
print("Provided Bad Answer: ", bad_answer.response['response'])
score, max_possible_score, eval, correct_answer = get_eval(task_id, bad_answer.toJSON())['result']
rprint(("\n").join(eval))
print(f"Score: {score}/{max_possible_score}")

Provided Bad Answer:  here is a summary


Does not error
✅ You successfully responded to the request.
You received 0.25 of 0.25 reward.
Does not take a long time
✅ You responded to the request in 0.44646668434143066.
You received 0.5 of 0.5 reward.
Return valid assistant response
❌ Your response was barely similar to what it should have been.
You received 0.75 of 3 reward.

Score: 1.5/3.75



 ## (E) (If applicable) Show the dataset that was used by the validator.
 > (1) Load the dataset 
 > 
 > (2) Show the first 10 samples of the dataset 
- Please link us to the original validator code where appropriate

In [18]:
# datasets - https://github.com/RogueTensor/bitagent_subnet/blob/main/bitagent/task_api/dataset.py
# only use QnA Dataset (not using ansible, api or summarization)

# from text dataset (open web text):
# hugging face data is pulled in sets of 3-100
# the data is provided as source data that may be valuable in answering the generated question
# then a single data entry is used to generate a question that relates to that data
# then the task is set up for eval
for i in range(10): 
    task, _ = get_task(*qna_task)
    display_markdown(f"""### QnA Task: #{i}
----------------------------
#### Prompt: 
{task.prompt}
#### Data:
{str(task.datas[0])[:200] + " ..."}""", raw=True)

# datasources - https://github.com/RogueTensor/bitagent_subnet/tree/main/bitagent/task_api/datasources
# only using conversation and tools

# tool calling from the tools data sources
# datasource is used to create the available tools list
# and then the conversation messages
for i in range(10):
    task, _ = get_task(*tool_call_task)
    display_markdown(f"""### Tool Calling Task: #{i}
----------------------------
#### Messages: 
{task.messages}
#### Tools:
{task.tools}""", raw=True)

# tool gen from the tools data sources
# datasource is used to create the prompt for expected tools
for i in range(10):
    task, _ = get_task(*tool_gen_task)
    display_markdown(f"""### Tool Gen Task: #{i}
----------------------------
#### Prompt: 
{task.prompt}""", raw=True)

# datasource is used to create the conversation messages
for i in range(10):
    task, _ = get_task(*conversation_task)
    display_markdown(f"""### Conversation Task: #{i}
----------------------------
#### Messages: 
{task.messages}""", raw=True)

### QnA Task: #0
----------------------------
#### Prompt: 
What was the outcome of the trial in West Germany for Hermine Braunsteiner Ryan, a former SS member and Nazi war criminal who was extradited from the United States in 1973?
#### Data:
{'source': 'bitagent.source.117799630451929260265424829358299386590', 'context': 'Hermine Braunsteiner Ryan (July 16, 1919 – April 19, 1999) was a German SS Helferin and female camp guard at Ravensbrü ...

### QnA Task: #1
----------------------------
#### Prompt: 
What was the name of the television station where Marrou joined the 10 o'clock news team in San Antonio in 1973?
#### Data:
{'source': 'bitagent.source.42662579660448188487433969951081273464', 'context': 'Chris Rene Marrou (born November 12, 1947) is former news anchor for KENS 5-TV in San Antonio, Texas from 1973 to 2009. ...

### QnA Task: #2
----------------------------
#### Prompt: 
Who was the editor of the "New Zealand Thoroughbred Racing Annual" for the 47th edition, published in 2018?
#### Data:
{'source': 'bitagent.source.185226731854978721404141283474147626414', 'context': "The Railway Stakes is a Group One New Zealand horse race. Held at Ellerslie on New Year's Day, it is held over 1200 me ...

### QnA Task: #3
----------------------------
#### Prompt: 
What are the most serious human rights issues regarding children's welfare in Lithuania according to the 2016 Country Reports on Human Rights Practices, and what are the main reasons for child abuse as reported by the Lithuanian Children's Rights Ombudsman Institution?
#### Data:
{'source': 'bitagent.source.208596608168655407594591205584232972047', 'context': 'Human rights in Lithuania have been the subject of much coverage. Lithuania scores 91 out of 100 in 2019 which is cons ...

### QnA Task: #4
----------------------------
#### Prompt: 
Given the text, which international scholarly periodical published three of the papers that were copied in the controversy surrounding Dr. Masoumeh Ebtekar's academic misconduct?
#### Data:
{'source': 'bitagent.source.224102324346492727496711596652779024347', 'context': 'Masoumeh Ebtekar (; born Masoumeh Ebtekar; 21 September 1960) is former Vice President of Iran for Women and Family Af ...

...

### Conversation Task: #5
----------------------------
#### Messages: 
messages=[ChatMessage(role=<ChatRole.USER: 'user'>, content='Let\'s delve into the process of crafting a PHP application that ranks your friends based on their performance in subject-specific exam questions and determines the overall top performer. Given an XML file containing each friend\'s unique study methods and subject area expertise, the question is: how can we evaluate and compare their scores?\n\nHere\'s a step-by-step guide on how to structure the XML data and write the PHP code to accomplish this task:\n\n1. **XML Data Structure**: The XML file might look something like this:\n\n```xml\n<friends>\n    <friend id="1">\n        <name>John Doe</name>\n        <techniques>\n            <technique id="1">Flash')]

### Conversation Task: #6
----------------------------
#### Messages: 
messages=[ChatMessage(role=<ChatRole.USER: 'user'>, content="What prompts numerous individuals to express disapproval towards Joe Biden?\n\nOr\n\nIn what ways does Joe Biden elicit criticism from a significant portion of the population?\n\nOr\n\nJoe Biden's leadership style has sparked controversy and criticism among many people. Why is this so?\n\nOr\n\nUnderstanding the reasons behind the dislike for Joe Biden is an intriguing topic. Let's delve into it.\n\nOr\n\nJoe Biden's popularity is a subject of debate among a substantial number of people. What makes him a divisive figure?\n\nOr\n\nThe question on everyone's lips: why does Joe Biden receive negative sentiments from so many people?\n\nEach of these rewrites aims")]

### Conversation Task: #7
----------------------------
#### Messages: 
messages=[ChatMessage(role=<ChatRole.USER: 'user'>, content='Here\'s a fresh take on your request:\n\nAre you up for the challenge of crafting an English sentence and a mathematical equation that embody the essence of the adjective "impeccable"? To elaborate, can you construct an English phrase that encapsulates the idea of "flawless" in linguistic terms, and follow it up with a Latex equation that symbolizes the concept mathematically? It\'s essential to approach this problem thoughtfully, as the equation should demand a degree of reasoning to decipher.\n\nBest of luck to you on this intriguing task!')]

### Conversation Task: #8
----------------------------
#### Messages: 
messages=[ChatMessage(role=<ChatRole.USER: 'user'>, content="Moving forward, let's imagine you're conversing with someone who's just uncovered the remarkable ability to transform into different forms. In this dynamic new role, respond to all my prompts as if you're this very individual, awestruck by her newly discovered shapeshifting powers.\n\nHere's a fun twist: Imagine you're a private investigator, unraveling the intricate mysteries of your own metamorphosis, while maintaining the same level of connection and engagement with me.\n\nLet's keep the conversation captivating, shall we?"), ChatMessage(role=<ChatRole.ASSISTANT: 'assistant'>, content="Greetings, my dear! My heart is pulsating with excitement to share my latest astonishment with you. I've always harbored a lively imagination, but now I comprehend it extends beyond that. I've found myself endowed with the ability to metamorphose!\n\nThe journey commenced a few days past when I found myself lost in a daydream about morphing into any animal I fancied. All of a sudden, I felt an unusual wave envelop me, and upon opening my eyes, I realized I had morphed into a resplendent owl. I was overwhelmed with awe and elation, and couldn't help but burst into laughter.\n\nSince then, I've been exploring the boundaries of"), ChatMessage(role=<ChatRole.USER: 'user'>, content="Guidance please, let's delve into it together. I'm eager to learn and understand more, so kindly lead the way. Let's unravel this mystery hand in hand.")]

### Conversation Task: #9
----------------------------
#### Messages: 
messages=[ChatMessage(role=<ChatRole.USER: 'user'>, content="What artistic genius graced the vaulted ceiling of the Sistine Chapel with his masterful strokes?\n\nOr, alternatively: Which illustrious painter took on the awe-inspiring task of adorning the Sistine Chapel's ceiling with his incredible talent?\n\nOr, in a more conversational tone: Who, among the list of great artists, has the honor of creating the breathtaking artwork that covers the Sistine Chapel's ceiling?\n\nIn any case, the question remains the same: Who is the gifted painter behind the stunning artwork on the Sistine Chapel's ceiling?")]

 ## (F) (If applicable) Demonstrate the use of any API and/or links to a frontend.


URL to app: https://gogoagent.ai \
API Endpoint: https://api.gogoagent.ai \
More documentation here - https://docs.google.com/document/d/1QVCzDu0eMmkdglD65F_Q_UjnCJauVEr62WgG8SgACt0

### Using the API endpoint

In [19]:
# Using the API
import json
import requests

api_endpoint = "https://api.gogoagent.ai/query_subnet"
# IF you are a validator on SN20 with an known IP, you'll have access
headers = {"Content-Type": "application/json"}
# IF you are NOT a validator on SN20, you can request a free API Key
# In the future, you can sign up for an API Key
# headers = {"Content-Type": "application/json", "Authorization": "<API KEY GOES HERE>"}

def tip_calculator(bill_amount, tip_percent):
    return bill_amount * tip_percent/100.

def another_calculator_for_summation(num1, num2):
    return num1 + num2

# Pose your user query
messages = [{"role": "user",
                    "content": "Need help calculating the tip, what is 10% tip on a bill totalling $100"}]

# Define the tools (see methods above)
tools = [{"name": "tip_calculator", "description": "Calculate the tip amount",
          "arguments": {"bill_amount": {"required": True, "type": "number",
                                        "description": "the bill amount in dollars"},
                        "tip_percent": {"required": True, "type": "number",
                                        "description": "the tip percentage as a whole number"},
                       }
        },
        {"name": "another_calculator_for_summation", "description": "Calculate the sum of two numbers",
          "arguments": {"num1": {"required": True, "type": "number",
                                 "description": "the first number for summation"},
                        "num2": {"required": True, "type": "number",
                                 "description": "the second number for summation"},
                       }
        }]

data = {"messages": messages, "tools": tools}
result = requests.post(api_endpoint, headers=headers, json=data)

display_markdown(f"""### Help Calculating a Tip using the SN20 API:
#### The Query:
{messages}
#### The Tools:
{tools}
### Results from TOP 5 miners:
""", raw=True)

for res in result.json():
    try:
        print(json.loads(res['response']['response'])[0])
    except:
        # the miner messed up the JSON or response
        pass

### Help Calculating a Tip using the SN20 API:
#### The Query:
[{'role': 'user', 'content': 'Need help calculating the tip, what is 10% tip on a bill totalling $100'}]
#### The Tools:
[{'name': 'tip_calculator', 'description': 'Calculate the tip amount', 'arguments': {'bill_amount': {'required': True, 'type': 'number', 'description': 'the bill amount in dollars'}, 'tip_percent': {'required': True, 'type': 'number', 'description': 'the tip percentage as a whole number'}}}, {'name': 'another_calculator_for_summation', 'description': 'Calculate the sum of two numbers', 'arguments': {'num1': {'required': True, 'type': 'number', 'description': 'the first number for summation'}, 'num2': {'required': True, 'type': 'number', 'description': 'the second number for summation'}}}]
### Results from TOP 5 miners:


{'role': 'tool call', 'content': {'name': 'tip_calculator', 'arguments': {'bill_amount': 100, 'tip_percent': 10}}}
{'role': 'tool call', 'content': {'name': 'tip_calculator', 'arguments': {'bill_amount': 100, 'tip_percent': 10}}}


#### Notice
Since the context was tip related, the tip calculator was passed back, but the summation calculator was not recommended for use.

From here, we can now call this method on our end - we have the Tip Calculator.

## Consent: Do you want this demo notebook to be public? Yes/No  

Yes